In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/official/ml_metadata/sdk-metric-parameter-tracking-for-custom-jobs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/official/ml_metadata/sdk-metric-parameter-tracking-for-custom-jobs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ml_metadata/sdk-metric-parameter-tracking-for-custom-jobs.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>  
</table>

# Vertex AI: Track parameters and metrics for custom training jobs

## Overview

This notebook demonstrates how to track metrics and parameters for Vertex AI custom training jobs, and how to perform detailed analysis using this data.

### Dataset

This example uses the Abalone Dataset. For more information about this dataset please visit: https://archive.ics.uci.edu/ml/datasets/abalone
### Objective

In this notebook, you will learn how to use Vertex AI SDK for Python to:

    * Track training parameters and prediction metrics for a custom training job.
    * Extract and perform analysis for all parameters and metrics within an Experiment.

### Costs 


This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install additional package dependencies not installed in your notebook environment.

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [2]:
! pip3 install -U tensorflow $USER_FLAG
! python3 -m pip install {USER_FLAG} google-cloud-aiplatform --upgrade
! pip3 install scikit-learn {USER_FLAG}


     |████████████████████████████████| 497.6 MB 7.3 kB/s eta 0:00:011| 172.1 MB 45.8 MB/s eta 0:00:08     |████████████████████▏           | 313.5 MB 47.8 MB/s eta 0:00:04     |████████████████████████▊       | 385.2 MB 52.6 MB/s eta 0:00:03     |█████████████████████████████▋  | 460.2 MB 50.0 MB/s eta 0:00:010.0 MB/s eta 0:00:01�█▏ | 469.7 MB 50.0 MB/s eta 0:00:01     |██████████████████████████████▎ | 470.8 MB 50.0 MB/s eta 0:00:01�█▍ | 472.2 MB 50.0 MB/s eta 0:00:01�█▌ | 473.5 MB 50.0 MB/s eta 0:00:016 MB 9.0 MB/s eta 0:00:033 MB 9.0 MB/s eta 0:00:02��██████████████████████ | 482.9 MB 9.0 MB/s eta 0:00:02███████▍| 488.3 MB 9.0 MB/s eta 0:00:02�██████████████████████████| 496.3 MB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 27.7 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 54.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 37.1 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 38.9 MB/s eta 0:00:01
     |████

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [3]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Select a GPU runtime

**Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select "Runtime --> Change runtime type > GPU"**

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  vertex-ai-dev


Otherwise, set your project ID here.

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

Set gcloud config to your project ID.

In [3]:
!gcloud config set project $PROJECT_ID

Updated property [core/project].


Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update



To take a quick anonymous survey, run:
  $ gcloud survey



#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [4]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebooks, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**


When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. Vertex AI runs
the code from this package. In this tutorial, Vertex AI also saves the
trained model that results from your job in the same bucket. Using this model artifact, you can then
create Vertex AI model and endpoint resources in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with Vertex AI.

In [5]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

In [6]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "-aip-" + TIMESTAMP

if REGION == "[your-region]":
    REGION = "us-central1"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [7]:
! gsutil mb -l $REGION $BUCKET_URI

Creating gs://vertex-ai-dev-aip-20220422060739/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [8]:
! gsutil ls -al $BUCKET_URI

### Import libraries and define constants

Import required libraries.


In [9]:
import pandas as pd
from google.cloud import aiplatform
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.python.keras.utils import data_utils

2022-04-22 06:08:05.168098: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-22 06:08:05.168138: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Initialize Vertex AI and set an _experiment_


Define experiment name.

In [10]:
EXPERIMENT_NAME = ""  # @param {type:"string"}

If EXEPERIMENT_NAME is not set, set a default one below:

In [11]:
if EXPERIMENT_NAME == "" or EXPERIMENT_NAME is None:
    EXPERIMENT_NAME = "my-experiment-" + TIMESTAMP

Initialize the *client* for Vertex AI.

In [12]:
aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET_URI,
    experiment=EXPERIMENT_NAME,
)

## Tracking parameters and metrics in Vertex AI custom training jobs

This example uses the Abalone Dataset. For more information about this dataset please visit: https://archive.ics.uci.edu/ml/datasets/abalone

In [13]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv
!gsutil cp abalone_train.csv {BUCKET_URI}/data/

gcs_csv_path = f"{BUCKET_URI}/data/abalone_train.csv"

--2022-04-22 06:08:19--  https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 173.194.203.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145915 (142K) [text/csv]
Saving to: ‘abalone_train.csv’

abalone_train.csv   100%[===================>] 142.50K  --.-KB/s    in 0.003s  

2022-04-22 06:08:19 (48.3 MB/s) - ‘abalone_train.csv’ saved [145915/145915]

Copying file://abalone_train.csv [Content-Type=text/csv]...
/ [1 files][142.5 KiB/142.5 KiB]                                                
Operation completed over 1 objects/142.5 KiB.                                    


### Create a managed tabular dataset from a CSV

A Managed dataset can be used to create an AutoML model or a custom model. 

In [14]:
ds = aiplatform.TabularDataset.create(display_name="abalone", gcs_source=[gcs_csv_path])

ds.resource_name

Creating TabularDataset


INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset


Create TabularDataset backing LRO: projects/931647533046/locations/us-central1/datasets/3337286071137337344/operations/5379081981415915520


INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/931647533046/locations/us-central1/datasets/3337286071137337344/operations/5379081981415915520


TabularDataset created. Resource name: projects/931647533046/locations/us-central1/datasets/3337286071137337344


...

To use this TabularDataset in another session:


INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:


ds = aiplatform.TabularDataset('projects/931647533046/locations/us-central1/datasets/3337286071137337344')


INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/931647533046/locations/us-central1/datasets/3337286071137337344')


'projects/931647533046/locations/us-central1/datasets/3337286071137337344'

### Write the training script

Run the following cell to create the training script that is used in the sample custom training job.

In [15]:
%%writefile training_script.py

import pandas as pd
import argparse
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

parser = argparse.ArgumentParser()
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--num_units', dest='num_units',
                    default=64, type=int,
                    help='Number of unit for first layer.')
args = parser.parse_args()
# uncomment and bump up replica_count for distributed training
# strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
# tf.distribute.experimental_set_strategy(strategy)

col_names = ["Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight", "Age"]
target = "Age"

def aip_data_to_dataframe(wild_card_path):
    return pd.concat([pd.read_csv(fp.numpy().decode(), names=col_names)
                      for fp in tf.data.Dataset.list_files([wild_card_path])])

def get_features_and_labels(df):
    return df.drop(target, axis=1).values, df[target].values

def data_prep(wild_card_path):
    return get_features_and_labels(aip_data_to_dataframe(wild_card_path))


model = tf.keras.Sequential([layers.Dense(args.num_units), layers.Dense(1)])
model.compile(loss='mse', optimizer='adam')

model.fit(*data_prep(os.environ["AIP_TRAINING_DATA_URI"]),
          epochs=args.epochs ,
          validation_data=data_prep(os.environ["AIP_VALIDATION_DATA_URI"]))
print(model.evaluate(*data_prep(os.environ["AIP_TEST_DATA_URI"])))

# save as Vertex AI Managed model
tf.saved_model.save(model, os.environ["AIP_MODEL_DIR"])

Writing training_script.py


### Launch a custom training job and track its trainig parameters on Vertex AI ML Metadata

In [16]:
job = aiplatform.CustomTrainingJob(
    display_name="train-abalone-dist-1-replica",
    script_path="training_script.py",
    container_uri="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest",
    requirements=["gcsfs==0.7.1"],
    model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest",
)

Start a new experiment run to track training parameters and start the training job. Note that this operation will take around 10 mins.

In [17]:
aiplatform.start_run("custom-training-run-1")  # Change this to your desired run name
parameters = {"epochs": 10, "num_units": 64}
aiplatform.log_params(parameters)

model = job.run(
    ds,
    replica_count=1,
    model_display_name="abalone-model",
    args=[f"--epochs={parameters['epochs']}", f"--num_units={parameters['num_units']}"],
)

Training script copied to:
gs://vertex-ai-dev-aip-20220422060739/aiplatform-2022-04-22-06:08:40.780-aiplatform_custom_trainer_script-0.1.tar.gz.


INFO:google.cloud.aiplatform.utils.source_utils:Training script copied to:
gs://vertex-ai-dev-aip-20220422060739/aiplatform-2022-04-22-06:08:40.780-aiplatform_custom_trainer_script-0.1.tar.gz.


Training Output directory:
gs://vertex-ai-dev-aip-20220422060739/aiplatform-custom-training-2022-04-22-06:08:40.949 


INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://vertex-ai-dev-aip-20220422060739/aiplatform-custom-training-2022-04-22-06:08:40.949 


No dataset split provided. The service will use a default split.


...

INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/2839724623975677952 current state:
PipelineState.PIPELINE_STATE_RUNNING


CustomTrainingJob run completed. Resource name: projects/931647533046/locations/us-central1/trainingPipelines/2839724623975677952


INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob run completed. Resource name: projects/931647533046/locations/us-central1/trainingPipelines/2839724623975677952


Model available at projects/931647533046/locations/us-central1/models/1354690684396765184


INFO:google.cloud.aiplatform.training_jobs:Model available at projects/931647533046/locations/us-central1/models/1354690684396765184


### Deploy Model and calculate prediction metrics

Deploy model to Google Cloud. This operation will take 10-20 mins.

In [18]:
endpoint = model.deploy(machine_type="n1-standard-4")

Creating Endpoint


INFO:google.cloud.aiplatform.models:Creating Endpoint


Create Endpoint backing LRO: projects/931647533046/locations/us-central1/endpoints/5579089125102845952/operations/8542860719643688960


INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/931647533046/locations/us-central1/endpoints/5579089125102845952/operations/8542860719643688960


Endpoint created. Resource name: projects/931647533046/locations/us-central1/endpoints/5579089125102845952


...

INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/931647533046/locations/us-central1/endpoints/5579089125102845952


Deploy Endpoint model backing LRO: projects/931647533046/locations/us-central1/endpoints/5579089125102845952/operations/3937930100657356800


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/931647533046/locations/us-central1/endpoints/5579089125102845952/operations/3937930100657356800


Endpoint model deployed. Resource name: projects/931647533046/locations/us-central1/endpoints/5579089125102845952


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/931647533046/locations/us-central1/endpoints/5579089125102845952


Once model is deployed, perform online prediction using the `abalone_test` dataset and calculate prediction metrics.

Prepare the prediction dataset.

In [19]:
def read_data(uri):
    dataset_path = data_utils.get_file("abalone_test.data", uri)
    col_names = [
        "Length",
        "Diameter",
        "Height",
        "Whole weight",
        "Shucked weight",
        "Viscera weight",
        "Shell weight",
        "Age",
    ]
    dataset = pd.read_csv(
        dataset_path,
        names=col_names,
        na_values="?",
        comment="\t",
        sep=",",
        skipinitialspace=True,
    )
    return dataset


def get_features_and_labels(df):
    target = "Age"
    return df.drop(target, axis=1).values, df[target].values


test_dataset, test_labels = get_features_and_labels(
    read_data(
        "https://storage.googleapis.com/download.tensorflow.org/data/abalone_test.csv"
    )
)

40960/37298 [================================] - 0s 0us/step


Perform online prediction.

In [20]:
prediction = endpoint.predict(test_dataset.tolist())
prediction

Prediction(predictions=[[11.7899103], [12.4608507], [8.1881876], [11.2659445], [9.37793827], [11.4920721], [7.93443251], [11.5312605], [12.6889906], [14.462513], [10.283576], [8.6949091], [11.2097692], [11.1072855], [11.0329885], [12.135272], [9.99304104], [10.8957777], [9.22797871], [13.7230711], [11.8929739], [8.09930325], [8.17502117], [11.703372], [10.0987978], [10.2399282], [8.10486317], [8.04834366], [8.49833679], [10.8826189], [6.79617643], [12.4286394], [8.68384647], [7.24311638], [8.84667301], [10.0152884], [12.3745852], [8.05304813], [10.7256517], [10.3701639], [9.95751286], [13.3061237], [7.56582785], [6.77601385], [10.26476], [7.04382181], [10.4061174], [8.60371876], [9.97134876], [8.31486607], [8.67987347], [10.5456343], [12.0501947], [9.28325939], [11.7336025], [14.3847513], [11.8823977], [11.4908943], [11.3797083], [9.26141644], [12.2167597], [10.9821768], [12.1926165], [9.27942944], [11.1379957], [8.5028553], [11.516098], [6.28786707], [11.7988539], [12.5438662], [9.129

Calculate and track prediction evaluation metrics.

In [21]:
mse = mean_squared_error(test_labels, prediction.predictions)
mae = mean_absolute_error(test_labels, prediction.predictions)

aiplatform.log_metrics({"mse": mse, "mae": mae})

### Extract all parameters and metrics created during this experiment.

In [22]:
aiplatform.get_experiment_df()

,experiment_name,run_name,param.epochs,param.num_units,metric.mse,metric.mae
0,my-experiment-20220422060739,custom-training-run-1,10.0,64.0,7.177111,1.906646


### View data in the Cloud Console

Parameters and metrics can also be viewed in the Cloud Console. 


In [23]:
print("Vertex AI Experiments:")
print(
    f"https://console.cloud.google.com/ai/platform/experiments/experiments?folder=&organizationId=&project={PROJECT_ID}"
)

Vertex AI Experiments:
https://console.cloud.google.com/ai/platform/experiments/experiments?folder=&organizationId=&project=vertex-ai-dev


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:
Training Job
Model
Cloud Storage Bucket

* Vertex AI Dataset
* Training Job
* Model
* Endpoint
* Cloud Storage Bucket


In [24]:
# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

#Delete dataset
ds.delete()

# Delete the training job
job.delete()

# Undeploy model from endpoint
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()


if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI

Deleting TabularDataset : projects/931647533046/locations/us-central1/datasets/3337286071137337344


INFO:google.cloud.aiplatform.base:Deleting TabularDataset : projects/931647533046/locations/us-central1/datasets/3337286071137337344


Delete TabularDataset  backing LRO: projects/931647533046/locations/us-central1/operations/2104965052317564928


INFO:google.cloud.aiplatform.base:Delete TabularDataset  backing LRO: projects/931647533046/locations/us-central1/operations/2104965052317564928


TabularDataset deleted. . Resource name: projects/931647533046/locations/us-central1/datasets/3337286071137337344


...

INFO:google.cloud.aiplatform.base:Deleting Model : projects/931647533046/locations/us-central1/models/1354690684396765184


Delete Model  backing LRO: projects/931647533046/locations/us-central1/operations/436381390376796160


INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/931647533046/locations/us-central1/operations/436381390376796160


Model deleted. . Resource name: projects/931647533046/locations/us-central1/models/1354690684396765184


INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/931647533046/locations/us-central1/models/1354690684396765184
